# TP3 : Parcours de graphes 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline 

In [ ]:
import numpy as np
import os
import scipy.sparse as sp
import re
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
import scipy.io as io
import string

# Exo 1 : prédiction de textes

Un texte est une séquence de caractères modélisable comme une série de tirages aléatoires selon une loi L à déterminer.

### Lecture des données

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train',remove=('headers', 'footers', 'quotes'))
n = newsgroups_train.filenames.shape[0]
corpus = newsgroups_train.data

In [ ]:
print corpus[0]

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


### Indexation des caractères

In [ ]:
caract =['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z',' ']
index = {}
for i in range(len(caract)):
    index[caract[i]] = i

## Modèle unigramme
Soit une séquence de symboles $x_1,...,x_t,...$. Le modèle unigramme considère chaque symbole $x_t$ comme issu d'un tirage multinomial de probabilité $(p_1,...,p_n)$ avec $\sum_i p_i =1$, où n est le nombre de symboles.

#### Fonction de tirage aléatoire 

In [ ]:
def tirage(p):
    # réalise un tirage multinomial à partir du vecteur de probabilités p
    v = np.random.multinomial(1,p, 1)
    return np.where(v==1)[1][0]

#### Fonction de "nettoyage" des caractères de ponctuation et des majuscules

In [ ]:
def nettoie(d):
    l = re.findall(r'[a-z \n]',d,re.I)
    r = string.join(l,'')
    r = string.replace(r,'\n',' ')
    #print d,'\n','-'*30,'\n',d_prim.lower()
    return r.lower()

In [ ]:
print nettoie(corpus[0])

i was wondering if anyone out there could enlighten me on this car i saw the other day it was a door sports car looked to be from the late s early s it was called a bricklin the doors were really small in addition the front bumper was separate from the rest of the body this is  all i know if anyone can tellme a model name engine specs years of production where this car is made history or whatever info you have on this funky looking car please email


### A Faire

1. A partir du corpus '20 newsgroups", calculez le vecteur de probabilité $(p_1,...,p_n)$ en fonction des fréquences d'apparition des différents caractères (pensez à utiliser la fonction de nettoyage ci-dessus).
2. Ensuite, vous utiliserez la fonction tirage ci-dessus pour générer une séquence de caractères obéissant à cette loi de probabilité.

## Modèle bigramme

Soit une séquence de symboles $x_1,...,x_t,...$. Le modèle bigramme considère que la probabilité d'apparition du symbole $x_t$ dépend du symbole précédent uniquement, soit $P(X_t=x_t|X_1=x_1,...,X_{t-1}=x_{t-1}) = P(X_t=x_t|X_{t-1}=x_{t-1}) $

Cette probabilité peut être représentée à l'aide d'un tableau bidimensionnel $(P_{ij})_{i,j= 1..n}$, avec $\sum_j P_{ij}=1$, où $P_{ij}$ représente la probabilité de choisir le symbole $j$ après le symbole $i$.

1. A partir du corpus '20 newsgroups", calculez la matrice de probabilité $((p_{11},...,p_{1n}), ..., (p_{n1}, ..., p_{nn}))$ en fonction des fréquences d'apparition des différents couples de caractères dans la base.

2. Ensuite, vous utiliserez la fonction tirage pour générer une séquence de caractères obéissant à cette loi de probabilité.

# Exo 2 : Enron database

La base Enron contient la liste des emails envoyés et reçus par les différents employés de la société "Enron". Cette liste permet de définir un grape des échanges de messages entre employés. A partir de la structure  de ce graphe, il est possible de calculer un score de "popularité" pour chaue employé selon le principe du "PageRank" vu en cours.

### Liste de employés

In [ ]:
employe = np.load('/content/drive/MyDrive/Text mining/employe.npy') #Chemin à modifier

In [ ]:
employe

array(['40enron@enron.com', '9069761@skytel.com', 'a..howard@enron.com',
       ..., 'zhiyong.wei@enron.com', 'zhiyun.yang@enron.com',
       'zimin.lu@enron.com'], dtype='|S39')

In [ ]:
len(employe)

2323

In [ ]:
employe[1191]

'kenneth.lay@enron.com'

In [ ]:
index = {}
for i in range(len(employe)):
    index[employe[i]] = i

In [ ]:
index['kenneth.lay@enron.com']

1191

### Lecture du graphe

In [ ]:
G = io.mmread('/content/drive/MyDrive/Text mining/M.mtx') #Chemin à modifier
G = G.tolil()

#### Affichage "matrice pleine" :

In [ ]:
G[:10,:10].todense()

matrix([[ 0.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  3.,  7.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.],
        [ 0.,  0.,  0.,  0., 62.,  0., 43.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  8.,  0.,  1.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 16.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

#### Affichage "matrice creuse" :

In [ ]:
sp.find(G[:10,:10])

(array([4, 2, 3, 2, 0, 4, 6, 7, 4, 6, 5, 7, 8], dtype=int32),
 array([0, 2, 2, 3, 4, 4, 4, 5, 6, 6, 7, 7, 8], dtype=int32),
 array([ 1.,  3.,  2.,  7.,  4.,  1., 62.,  8.,  1., 43.,  7.,  1., 16.]))

#### Liste des destinataires et nombre de messages envoyés pour l'employé 1191 :


In [ ]:
print(employe[1191])
T=sp.find(G[1191,:])

kenneth.lay@enron.com


In [ ]:
T[1].astype(float)

array([2.000e+00, 3.000e+00, 4.000e+00, 5.000e+00, 7.000e+00, 1.300e+01,
       1.500e+01, 3.900e+01, 6.600e+01, 7.900e+01, 9.500e+01, 1.020e+02,
       1.220e+02, 1.390e+02, 1.500e+02, 1.510e+02, 1.530e+02, 1.600e+02,
       1.610e+02, 1.660e+02, 1.670e+02, 1.680e+02, 1.760e+02, 1.850e+02,
       1.960e+02, 1.980e+02, 2.040e+02, 2.160e+02, 2.240e+02, 2.340e+02,
       2.410e+02, 2.420e+02, 2.460e+02, 2.510e+02, 2.530e+02, 2.680e+02,
       2.710e+02, 2.720e+02, 2.760e+02, 2.860e+02, 2.950e+02, 2.990e+02,
       3.060e+02, 3.090e+02, 3.100e+02, 3.130e+02, 3.140e+02, 3.220e+02,
       3.240e+02, 3.370e+02, 3.400e+02, 3.420e+02, 3.440e+02, 3.470e+02,
       3.530e+02, 3.610e+02, 3.650e+02, 3.670e+02, 3.770e+02, 3.870e+02,
       3.880e+02, 3.900e+02, 3.910e+02, 3.930e+02, 3.950e+02, 3.970e+02,
       4.030e+02, 4.140e+02, 4.200e+02, 4.290e+02, 4.330e+02, 4.360e+02,
       4.390e+02, 4.490e+02, 4.720e+02, 4.730e+02, 4.770e+02, 4.780e+02,
       4.800e+02, 4.870e+02, 4.970e+02, 5.330e+02, 

#### Tirage multinomial (idem exercice précédent)

In [ ]:
def tirage(x):
    v = np.random.multinomial(1,x, 1)
    return np.where(v==1)[1][0]

In [ ]:
k=[0.5,0.25,2000]

In [ ]:
tirage(k)

0

**PROBLEME**

1. Calculer la matrice de transition $P$ à partir du graphe de liens en tenant compte des valeurs:
  $$\forall (i,j), P_{ij} = \frac{G_{ij}} {\sum_k G_{ik}}$$
1. Implémenter l'algorithme PageRank simplifié (sans mise à jour de la matrice de transition) :
  * Initialiser le vecteur $\boldsymbol{x}$ à la valeur $(1/n,1/n, ..., 1/n)$ 
  * Pour chaque employé visité $i$ :
    - choisir un employé $j$ au hasard parmi les liens sortants
    - Mettre à jour le score $x_j$ de la page $j$, i.e. $$ x_j \leftarrow q \sum_{i:i\rightarrow j} P_{ij} x_i + \frac{1-q}{n}$$
    - $i \leftarrow j$
  
  (q = 0.85)
1. Comment faire pour gérér les noeuds "cul-de-sac"?
1. Quelle est le score et la fonction des 10 employés les plus "populaires"?
(voir : http://www.inf.ed.ac.uk/teaching/courses/tts/assessed/roles.txt)
1. Quel est le score et le rang de Kenneth Lay?


#Matrice de transition

In [ ]:
def matrice_transition(G):
 P=np.zeros(G.shape)
 for i in range(G.shape[0]):
    R=G.getrow(i)
    I=sp.find(R)
    somme= sum(I[2].astype(float))
    for j in range(len(I[1])):
     k=I[1][j]
     P[i,k]=float(I[2][j])/float(somme)
 return P

In [ ]:
M=matrice_transition(G)

In [ ]:
M

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01060071, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.00430108, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.00815851, 0.00582751,
        0.01165501]])

#Algorithm de Page rank

In [ ]:
import random
random.seed(100)

In [ ]:
def Random_j(i): #Cette fonction permet de choisir aléatoirement un autre employé j sachant que l'amployé visité est i; elle joue le rôle de la fonction tirage définie plus haut
  L=G.getrow(i)
  outs=sp.find(L)[1]
  if outs.size==0:
    return np.random.randint(2323)
  else:
    return random.choice(outs)

def PageRank(G,it_max,q):
  P=matrice_transition(G)
  (n,m)=P.shape
  X=np.ones(n)/n
  i=np.random.randint(n)
  for iter in range(it_max):
    j=random_j(i)
    entrant_j=sp.find(G.getcol(j))[0]
    s=0
    for k in entrant_j:
      s=s+X[k]*P[k][j]
    X[j]=q*s+(1-q)*(1/n)
  return x

Comme nous pouvons le voir, il  faut préciser le nombre d'itérations maximale it_max.

In [ ]:
x=PageRank(G,2000,0.85)

In [ ]:
len(x)

2323

#Les Noeuds "cul-de-sac"


Pour les noeuds "cul-de-sac", il faut créer des liens sortant vers toutes les autres pages.

Du point de vue de la matrice, cela revient à trouver les lignes qui n’ont que des 0 et les remplacer par des lignes avec 1/(n-1) dans toutes les cases sauf sur la diagonale (lien d’un mail d'un employé vers lui-même). 

Pourquoi 1/(n-1) ? Tout simplement par ce qu’il y a n employés et qu’on considère que l’employé enverra son mail aléatoirement sur n’importe quel mail d'un autre employé (n(tous les employés) – 1 (son propre mail) = n-1).

#les scores des 10 premiers employés et leurs fonctions

In [ ]:
dic={}
for i in range(len(x)):
  dic[i]=x[i]

In [ ]:
dic_sorted=sorted(dic.items(), key=lambda x: x[1],reverse=True)

In [ ]:
dic_sorted[:10]

[(2148, 0.009664806004738472),
 (2094, 0.004954289710426263),
 (1373, 0.0037412681462557907),
 (1840, 0.002728985493337708),
 (1584, 0.0017568845266343626),
 (1160, 0.0015521734600351005),
 (1450, 0.0012704932606570894),
 (2124, 0.001255112542719331),
 (2191, 0.0011255072797935002),
 (1465, 0.0009229780514692583)]

In [ ]:
for i in range(len(dic_sorted[:10])):
  print(employe[dic_sorted[:10][i][0]])
  

tana.jones@enron.com
steven.kean@enron.com
louise.kitchen@enron.com
richard.shapiro@enron.com
michelle.cash@enron.com
kay.mann@enron.com
mark.haedicke@enron.com
susan.scott@enron.com
tim.belden@enron.com
mark.taylor@enron.com


Tana Jones: N/A

Steven Kean: Vice President - Vice President & Chief of Staff

Louise Kitchen: President  -  Enron Online

Richard Shapiro: Vice President  -   Regulatory Affairs

Michelle Cash: N/A

Kay Mann: Employee

Mark Haedicke: Managing Director  Legal Department

Susan Scott: N/A

Tim Belden:

Mark Taylor: Employee

#Score et rang de Kenneth Lay

In [ ]:
score_kenneth_Lay=dic[index['kenneth.lay@enron.com']]
print("le score de Kenneth Lay est {}".format(score_kenneth_Lay))

le score de Kenneth Lay est 0.00043


In [ ]:
index['kenneth.lay@enron.com']

1191

In [ ]:
index_kenneth_Lay=[dic_sorted.index(tupl) for tupl in dic_sorted if tupl[0] ==index['kenneth.lay@enron.com']]
rang_kenneth_Lay=index_kenneth_Lay[0]+1
print("le rang de Kenneth Lay est {}".format(rang_kenneth_Lay))

le rang de Kenneth Lay est 1132
